# Argumentative Causal Discovery

Notebook collecting results for causal discovery algorithm ABAPC presented in the Argumentative Causal Discovery paper (Russo, Rapberger and Toni (2024))

In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.insert(0,'../../')
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 2000)
sys.path.append('../../ArgCausalDisco/utils/')
from ArgCausalDisco.utils.plotting import *
print(sys.path)

save_figs = True
debug = False
datasets = ['cancer', 'earthquake', 'survey', 'asia']
dags_nodes_map = {'asia':8, 'cancer':5, 'earthquake':5, 'sachs':11, 'survey':6, 'alarm':37, 'child':20, 'insurance':27, 'hailfinder':56, 'hepar2':70}
dags_arcs_map = {'asia':8, 'cancer':4, 'earthquake':4, 'sachs':17, 'survey':6, 'alarm':46, 'child':25, 'insurance':52, 'hailfinder':66, 'hepar2':123}
methods = ['Random', 'FGS', 'NOTEARS-MLP', 'MPC', 'ABAPC (Ours)', 'ABASP (New)']
names_dict = {'fgs':'FGS', 'nt':'NOTEARS-MLP', 'mpc':'MPC', 'random':'Random', 'abapc':'ABAPC (Ours)', 'ABASP (New)':'ABASP (New)'}
symbols_dict = {'abapc':'triangle-down-dot','fgs':'triangle-up-dot','nt':'pentagon-dot','mpc':'hexagon2-dot', 'random':'x'}  
colors_dict = {'abapc':sec_blue,'fgs':sec_orange,'nt':main_purple,'mpc':main_green,'random':'grey', 'ABASP (New)':'black'}
version = 'bnlearn_50rep' ## for 5000 samples
# version = 'bnlearn_dag_v5_2000' ## for 2000 samples

['../../', '/opt/homebrew/Caskroom/miniconda/base/envs/aba-env/lib/python312.zip', '/opt/homebrew/Caskroom/miniconda/base/envs/aba-env/lib/python3.12', '/opt/homebrew/Caskroom/miniconda/base/envs/aba-env/lib/python3.12/lib-dynload', '', '/opt/homebrew/Caskroom/miniconda/base/envs/aba-env/lib/python3.12/site-packages', '../../ArgCausalDisco/utils/']


## Main Paper Results

### CPDAG Evaluation

In [2]:
version_cpdag = version+'_cpdag'
new = np.load('./ABASP.npy', allow_pickle=True)
old = np.load(f"../../ArgCausalDisco/results/stored_results_{version}_cpdag.npy", allow_pickle=True)
all_sum = pd.DataFrame(np.vstack((old, new)), 
                       columns=['dataset', 'model', 'elapsed_mean', 'elapsed_std', 'nnz_mean', 'nnz_std', 
                                'fdr_mean', 'fdr_std', 'tpr_mean', 'tpr_std', 'fpr_mean', 'fpr_std', 
                                'precision_mean', 'precision_std', 'recall_mean', 'recall_std',
                                'F1_mean', 'F1_std', 'shd_mean', 'shd_std', 
                                'SID_low_mean', 'SID_low_std', 'SID_high_mean', 'SID_high_std'
                                ])

all_sum['n_edges'] = all_sum['dataset'].map(dags_arcs_map)
all_sum['n_nodes'] = all_sum['dataset'].map(dags_nodes_map)
for var in ['shd','SID_low','SID_high']:
    all_sum['p_'+var+'_mean'] = all_sum[var+'_mean'].astype(float)/all_sum['n_edges'].astype(int)
    all_sum['p_'+var+'_std'] = all_sum[var+'_std'].astype(float)/all_sum['n_edges'].astype(int)
all_sum['dataset'] = [a.upper() for a in all_sum["dataset"].astype(str)]
all_sum['dataset'] = all_sum['dataset'] +np.repeat("<br> |V|=",len(all_sum))+ all_sum["n_nodes"].astype(str)+np.repeat(", |E|=",len(all_sum))+\
                     all_sum["n_edges"].astype(str)

double_bar_chart_plotly(all_sum, ['p_SID_low','p_SID_high'], names_dict, colors_dict, methods, save_figs=save_figs, output_name="./Fig.2_SID_cpdag.html", debug=False, range_y1=[0,6], range_y2=[0,6])#